> Version: _0.0.3_

## Basic Usage

`python-hn` is a small library to access [Hacker New's Search API](https://hn.algolia.com/api) powered by [Algolia](https://www.algolia.com).

It's designed to be simple to use and pythonic, but also provide the full power of Search API.

Install with `pip`:

In [99]:
!pip install python-hn

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Quick example

The main function of the library is `search_by_date`, which accepts multiple parameters; among them, is a query string containing the phrase to search for.

In [100]:
from hn import search_by_date

In [101]:
results = search_by_date('python')

By default, all the different "post types" will be included: stories, comments, polls, etc.

#### > Util function for prettier outputs:

In [102]:
import html
import tabulate

from itertools import islice
from IPython.display import HTML, display


def output_results(results, count=None, url=True, post_type=True, text_length=25):
    if count:
        results = islice(results, count)
    lines = []
    header = ['ID']
    if post_type:
        header += ['Post Type']
    header += ['Text']
    if url:
        header += ['URL']

    for post in results:
        line = [post['objectID']]
        if post_type:
            line += [post['_tags'][0].title()]
        
        text = html.unescape(
            post['title'] or str(post['comment_text'] or '')[:text_length] + '...')
        line += [text]
        if url:
            link = "https://news.ycombinator.com/item?id={}".format(post['objectID'])
            line += [f"<a href={link}>{link}</a>"]
        lines.append(line)
    return display(HTML(tabulate.tabulate(lines, headers=header, tablefmt='html')))

In [103]:
output_results(results, 10)

ID,Post Type,Text,URL
18608592,Story,MIT AI: Python with Guido van Rossum,https://news.ycombinator.com/item?id=18608592
18608502,Comment,Here's another artic...,https://news.ycombinator.com/item?id=18608502
18608367,Comment,"Mostly Go these days, sti...",https://news.ycombinator.com/item?id=18608367
18608027,Comment,Beside python has pretty ...,https://news.ycombinator.com/item?id=18608027
18607997,Comment,High level dynamic langua...,https://news.ycombinator.com/item?id=18607997
18607971,Comment,"Oddly enough, so does Cyt...",https://news.ycombinator.com/item?id=18607971
18607880,Comment,Yeah I'm mostly comi...,https://news.ycombinator.com/item?id=18607880
18607831,Comment,Coincidentally I started ...,https://news.ycombinator.com/item?id=18607831
18607679,Comment,There's a few other ...,https://news.ycombinator.com/item?id=18607679
18607599,Comment,That's why I use org...,https://news.ycombinator.com/item?id=18607599



As you can see, we're receiving both results for stories and comments.

### Searching by author

The second parameter of the `search_by_date` function is the author's username of the post (story or comment). Example:

In [104]:
# 'pg' is Paul Graham: https://news.ycombinator.com/user?id=pg
results = search_by_date('lisp', 'pg')

In [105]:
output_results(results, 10)

ID,Post Type,Text,URL
7446596,Comment,This was all the code it ...,https://news.ycombinator.com/item?id=7446596
7261591,Story,Source of the recent outagelet,https://news.ycombinator.com/item?id=7261591
7128860,Comment,Why doesn't the arti...,https://news.ycombinator.com/item?id=7128860
6705398,Comment,I thought this would be a...,https://news.ycombinator.com/item?id=6705398
6557282,Comment,"Has ""Lisp hacker&quo...",https://news.ycombinator.com/item?id=6557282
6400769,Comment,It's a real person. ...,https://news.ycombinator.com/item?id=6400769
6210178,Comment,Wow is this list good. I...,https://news.ycombinator.com/item?id=6210178
5575255,Story,Danny Sullivan's Twitter list about Watertown,https://news.ycombinator.com/item?id=5575255
5419609,Comment,Unfortunately I can't nam...,https://news.ycombinator.com/item?id=5419609
5376251,Comment,It happens. There are in...,https://news.ycombinator.com/item?id=5376251


### Filtering results by post type

You can pass multiple boolean parameters to filter the post types: `stories`, `comments`, `show_hn`, `ask_hn`, `front_page`, `polls`, `pollopt`.

In [106]:
results = search_by_date('python', stories=True)

In [107]:
output_results(results, 10)

ID,Post Type,Text,URL
18608592,Story,MIT AI: Python with Guido van Rossum,https://news.ycombinator.com/item?id=18608592
18606963,Story,Azure Functions gets better for Python and JavaScript developers,https://news.ycombinator.com/item?id=18606963
18606914,Story,Ask HN: What programming language to choose for coding interviews?,https://news.ycombinator.com/item?id=18606914
18606907,Story,Best Practices for Using Functional Programming in Python,https://news.ycombinator.com/item?id=18606907
18605794,Story,A Simple Application of Probabilistic Programming with PyMC3 in Python,https://news.ycombinator.com/item?id=18605794
18604364,Story,Show HN: Packagr.app – a cloud hosted PyPI server for Python developers,https://news.ycombinator.com/item?id=18604364
18604175,Story,How to determine which commands failed/succeeded in your Bash_History file?,https://news.ycombinator.com/item?id=18604175
18604165,Story,"Do “good” solutions to things like E2EE, authentication actually exist?",https://news.ycombinator.com/item?id=18604165
18603813,Story,Deciphering Python: How to Use Abstract Syntax Trees (AST) to Understand Code,https://news.ycombinator.com/item?id=18603813
18600537,Story,Getting started with graph analysis in Python with pandas and networkx,https://news.ycombinator.com/item?id=18600537


Also filtering by author:

In [108]:
results = search_by_date('lisp', author='pg', stories=True)

In [109]:
output_results(results, 10)

ID,Post Type,Text,URL
7261591,Story,Source of the recent outagelet,https://news.ycombinator.com/item?id=7261591
5575255,Story,Danny Sullivan's Twitter list about Watertown,https://news.ycombinator.com/item?id=5575255
5092711,Story,What we discovered about InstallMonetizer,https://news.ycombinator.com/item?id=5092711
2463285,Story,Andreessen Horowitz Leads $1.75M Round In Freebie Marketplace Listia,https://news.ycombinator.com/item?id=2463285
878576,Story,Final Startup School Speaker List (Evan and Biz now coming),https://news.ycombinator.com/item?id=878576
842685,Story,Doug McIlroy: McCarthy Presents Lisp,https://news.ycombinator.com/item?id=842685
93526,Story,"New: Best Comments (via ""Lists"" Link at Bottom)",https://news.ycombinator.com/item?id=93526
35935,Story,Arrington: The FCC Needs To Listen To Google,https://news.ycombinator.com/item?id=35935
7687,Story,"My Pairwise Test Results (not bad, but got the book part wrong)",https://news.ycombinator.com/item?id=7687


Filtering by query, author and including both stories and comments:

In [110]:
results = search_by_date('lisp', author='pg', stories=True, comments=True)

In [111]:
output_results(results, 10)

ID,Post Type,Text,URL
7446596,Comment,This was all the code it ...,https://news.ycombinator.com/item?id=7446596
7261591,Story,Source of the recent outagelet,https://news.ycombinator.com/item?id=7261591
7128860,Comment,Why doesn't the arti...,https://news.ycombinator.com/item?id=7128860
6705398,Comment,I thought this would be a...,https://news.ycombinator.com/item?id=6705398
6557282,Comment,"Has ""Lisp hacker&quo...",https://news.ycombinator.com/item?id=6557282
6400769,Comment,It's a real person. ...,https://news.ycombinator.com/item?id=6400769
6210178,Comment,Wow is this list good. I...,https://news.ycombinator.com/item?id=6210178
5575255,Story,Danny Sullivan's Twitter list about Watertown,https://news.ycombinator.com/item?id=5575255
5419609,Comment,Unfortunately I can't nam...,https://news.ycombinator.com/item?id=5419609
5376251,Comment,It happens. There are in...,https://news.ycombinator.com/item?id=5376251


Other examples:

In [112]:
results = search_by_date('lisp', author='pg', polls=True)

In [113]:
output_results(results, 10)

ID,Post Type,Text,URL
1185896,Poll,Poll: Ask or Leaders in Top Bar?,https://news.ycombinator.com/item?id=1185896


In [114]:
results = search_by_date('python', show_hn=True)

In [115]:
output_results(results, 10)

ID,Post Type,Text,URL
18604364,Story,Show HN: Packagr.app – a cloud hosted PyPI server for Python developers,https://news.ycombinator.com/item?id=18604364
18600380,Story,Show HN: StrictYAML 1.0 released: parse and validate YAML sanely in python,https://news.ycombinator.com/item?id=18600380
18599179,Story,Show HN: Enophp – PHP library for the eno notation language,https://news.ycombinator.com/item?id=18599179
18583789,Story,"Show HN: PowerSong, a platform to see which songs make you run or ride faster",https://news.ycombinator.com/item?id=18583789
18576532,Story,Show HN: A sum-type decorator for python,https://news.ycombinator.com/item?id=18576532
18544818,Story,Show HN: CV Compiler – A Python-based tool to improve tech resumes,https://news.ycombinator.com/item?id=18544818
18529273,Story,Show HN: Packagr.app – a cloud hosted PyPI server for Python developers,https://news.ycombinator.com/item?id=18529273
18516123,Story,Show HN: Machine Learning on JVM or a piece of cake,https://news.ycombinator.com/item?id=18516123
18509313,Story,Show HN: Efficient environment variables management and typing for python,https://news.ycombinator.com/item?id=18509313
18502878,Story,Show HN: Packagr.app – a cloud hosted PyPI server for Python developers,https://news.ycombinator.com/item?id=18502878


### Tags

HN Search API supports "tags", that can be logically combined to create complex expressions. There are 3 types of tags included in `python-hn`:

* `PostType`: with options `story`, `comment`, `poll`, `pollopt`, `show_hn`, `ask_hn`, `front_page`.
* `Author`: receives the username as param (`Author('pg')`).
* `StoryID`: receives the story id (`StoryID('6902129')`)

`PostType`s also have aliases that can be used instead of writing the full tag.

In [116]:
from hn import PostType, Author, StoryID
from hn.tags import Story, Comment, AskHN, ShowHN

In [117]:
PostType('story') == Story

True

The power of the tags arise when we combine them logically with `or` and `and` expressions. For example, you could ask for posts that are either _Ask HN_ or _Show HN_. To combine tags with an `or` expression, the operator is `|`:

In [118]:
tags = PostType('ask_hn') | PostType('show_hn')

In [119]:
results = search_by_date('python', tags=tags)

In [120]:
output_results(results, 10)

ID,Post Type,Text,URL
18606914,Story,Ask HN: What programming language to choose for coding interviews?,https://news.ycombinator.com/item?id=18606914
18604364,Story,Show HN: Packagr.app – a cloud hosted PyPI server for Python developers,https://news.ycombinator.com/item?id=18604364
18600380,Story,Show HN: StrictYAML 1.0 released: parse and validate YAML sanely in python,https://news.ycombinator.com/item?id=18600380
18600209,Story,Ask HN: What are some simple things I can build by myself?,https://news.ycombinator.com/item?id=18600209
18599811,Story,Ask HN: Building a side project that makes money. Where to start?,https://news.ycombinator.com/item?id=18599811
18599179,Story,Show HN: Enophp – PHP library for the eno notation language,https://news.ycombinator.com/item?id=18599179
18598025,Story,Ask HN: R or other non-Python languages for data science work,https://news.ycombinator.com/item?id=18598025
18583789,Story,"Show HN: PowerSong, a platform to see which songs make you run or ride faster",https://news.ycombinator.com/item?id=18583789
18576532,Story,Show HN: A sum-type decorator for python,https://news.ycombinator.com/item?id=18576532
18576231,Story,Ask HN: What's up with Data Scientist in 3 months programs?,https://news.ycombinator.com/item?id=18576231


Using aliases:

In [121]:
tags = AskHN | ShowHN

In [122]:
results = search_by_date('python', tags=tags)

In [123]:
output_results(results, 10)

ID,Post Type,Text,URL
18606914,Story,Ask HN: What programming language to choose for coding interviews?,https://news.ycombinator.com/item?id=18606914
18604364,Story,Show HN: Packagr.app – a cloud hosted PyPI server for Python developers,https://news.ycombinator.com/item?id=18604364
18600380,Story,Show HN: StrictYAML 1.0 released: parse and validate YAML sanely in python,https://news.ycombinator.com/item?id=18600380
18600209,Story,Ask HN: What are some simple things I can build by myself?,https://news.ycombinator.com/item?id=18600209
18599811,Story,Ask HN: Building a side project that makes money. Where to start?,https://news.ycombinator.com/item?id=18599811
18599179,Story,Show HN: Enophp – PHP library for the eno notation language,https://news.ycombinator.com/item?id=18599179
18598025,Story,Ask HN: R or other non-Python languages for data science work,https://news.ycombinator.com/item?id=18598025
18583789,Story,"Show HN: PowerSong, a platform to see which songs make you run or ride faster",https://news.ycombinator.com/item?id=18583789
18576532,Story,Show HN: A sum-type decorator for python,https://news.ycombinator.com/item?id=18576532
18576231,Story,Ask HN: What's up with Data Scientist in 3 months programs?,https://news.ycombinator.com/item?id=18576231


To combine tags with an `and` expression, the operator is `&`. For example, all the posts that are **comments** of the story ID [7261591](https://news.ycombinator.com/item?id=7261591):

In [124]:
tags = PostType('comment') & StoryID('7261591')

In [125]:
results = search_by_date(tags=tags)

In [126]:
output_results(results, 10)

ID,Post Type,Text,URL
7298300,Comment,"Sorry, this is uneducated...",https://news.ycombinator.com/item?id=7298300
7269494,Comment,I was able to keep a 10k&...,https://news.ycombinator.com/item?id=7269494
7269489,Comment,"Hmm, that sounds like qui...",https://news.ycombinator.com/item?id=7269489
7268211,Comment,You just need enough core...,https://news.ycombinator.com/item?id=7268211
7266280,Comment,Serving 403s to w00tw00ts...,https://news.ycombinator.com/item?id=7266280
7265942,Comment,"Things like ""Top 10 ...",https://news.ycombinator.com/item?id=7265942
7265025,Comment,Right ... websites with t...,https://news.ycombinator.com/item?id=7265025
7264560,Comment,In this case using nginx ...,https://news.ycombinator.com/item?id=7264560
7262931,Comment,I'm not sure I parti...,https://news.ycombinator.com/item?id=7262931
7262890,Comment,"Interestingly, distinguis...",https://news.ycombinator.com/item?id=7262890


### Filtering by _date_, _points_ or _number of comments_

It is also possible to pass different filters to restrain the search by creation date, number of points or comments. Namely, the parameters are:

* Creation Date: `created_at`
* Points: `points`
* Number of comments: `num_comments`

They accept `>, <, >=, <=` operators with a syntax similar to Django's:

* `lt` (`<`): Lower than. Example `ponts__lt=100`
* `lte` (`<=`): Lower than or equals to. Example `ponts__lte=100`
* `gt` (`>`): Greater than. Example `created_at__gt='2018'` (created after 2018-01-01).
* `gte` (`>=`): Greater than or equals to. Example `num_comments__gte=50`.

A few more examples using filters:

##### All post types created after October 1st, 2018

The filter is `created_at__gt`, `gt` is the `>` operator. The `created_at` filter will try to parse dates automatically (eg: `2018` is interpreted as `2018-01-01`).

In [127]:
results = search_by_date(stories=True, created_at__gt='2018-10')

In [128]:
output_results(results, 10)

ID,Post Type,Text,URL
18608634,Story,A new app can detect anemia from only a photo of nails,https://news.ycombinator.com/item?id=18608634
18608633,Story,Product development platform powered by AI,https://news.ycombinator.com/item?id=18608633
18608627,Story,Most software architecture diagrams are useless,https://news.ycombinator.com/item?id=18608627
18608621,Story,Hamiltonian Monte Carlo explained,https://news.ycombinator.com/item?id=18608621
18608615,Story,Danielle Morill on Mattermark's “premature scaling”,https://news.ycombinator.com/item?id=18608615
18608612,Story,Generating Fake Data in Development to Populate Your Database,https://news.ycombinator.com/item?id=18608612
18608604,Story,GQLify – One GraphQL for All APIs,https://news.ycombinator.com/item?id=18608604
18608603,Story,New theory could explain missing 95 percent of the cosmos,https://news.ycombinator.com/item?id=18608603
18608600,Story,Code-based design tool,https://news.ycombinator.com/item?id=18608600
18608592,Story,MIT AI: Python with Guido van Rossum,https://news.ycombinator.com/item?id=18608592


##### Created after October 1st, 2017 and before January 1st 2018

In [129]:
results = search_by_date(stories=True, created_at__gt='2017-10', created_at__lt='2018')

In [130]:
output_results(results, 10)

ID,Post Type,Text,URL
16043719,Story,RAM-less Buffers,https://news.ycombinator.com/item?id=16043719
16043643,Story,Profitable remote job site built with single PHP file (4.5K lines of code),https://news.ycombinator.com/item?id=16043643
16043612,Story,Subreddit Gender Ratios,https://news.ycombinator.com/item?id=16043612
16043585,Story,Core Infrastructure Initiative Best Practices Badge,https://news.ycombinator.com/item?id=16043585
16043578,Story,IOHIDeous OS X Local Kernel Vulnerability,https://news.ycombinator.com/item?id=16043578
16043561,Story,Dave Barry's 2017 Year in Review,https://news.ycombinator.com/item?id=16043561
16043557,Story,New Year’s Resolutions and the Science of Willpower (2015),https://news.ycombinator.com/item?id=16043557
16043554,Story,Top Cryptocurrencies or Altcoins to have in 2018,https://news.ycombinator.com/item?id=16043554
16043548,Story,The Google Book,https://news.ycombinator.com/item?id=16043548
16043542,Story,AI and Deep Learning in 2017 – A Year in Review,https://news.ycombinator.com/item?id=16043542


##### A few more examples:

In [131]:
# Stories with *exactly* 1000 points
results = search_by_date(stories=True, points=1000)

In [132]:
output_results(results, 10)

ID,Post Type,Text,URL
14090063,Story,IndieHackers.com acquired by Stripe,https://news.ycombinator.com/item?id=14090063
13489156,Story,First they came for the Iranians,https://news.ycombinator.com/item?id=13489156


In [133]:
# Stories including "Python" with more than 1000 points
results = search_by_date('python', stories=True, points__gt=1000)

In [134]:
output_results(results, 10)

ID,Post Type,Text,URL
17515492,Story,“I'm basically giving myself a permanent vacation from being BDFL”,https://news.ycombinator.com/item?id=17515492
13319904,Story,Grumpy: Go running Python,https://news.ycombinator.com/item?id=13319904


In [135]:
# Comments with more than 50 points
results = search_by_date('python', comments=True, points__gt=50)

In [139]:
output_results(results, 10)

ID,Post Type,Text,URL
18600293,Story,Fortnite dev launches Epic Games Store that takes 12% of revenue,https://news.ycombinator.com/item?id=18600293
18600220,Story,"Ask HN: What to do after $8M (all cash, post tax) exit?",https://news.ycombinator.com/item?id=18600220
18599728,Story,"What we learned from 3 years of bra engineering, and what's next",https://news.ycombinator.com/item?id=18599728
18599200,Story,Show HN: Dark Reader extension – Dark mode for every website,https://news.ycombinator.com/item?id=18599200
18599147,Story,Algorithmic auditors are exposing employee expense fraud,https://news.ycombinator.com/item?id=18599147
18598989,Story,Leaving NYC for Nashville,https://news.ycombinator.com/item?id=18598989
18598544,Story,Absolute Beginner's Guide to Emacs (2012),https://news.ycombinator.com/item?id=18598544
18598448,Story,"Photons, Quasars and the Possibility of Free Will",https://news.ycombinator.com/item?id=18598448
18596841,Story,A Not-Called Function Can Cause a 5X Slowdown,https://news.ycombinator.com/item?id=18596841
18596450,Story,Udacity lays off 125 people in global strategy shift,https://news.ycombinator.com/item?id=18596450


In [137]:
# Stories with 100 comments or more
results = search_by_date(stories=True, num_comments__gt=100)

In [138]:
output_results(results, 10)

ID,Post Type,Text,URL
18605709,Story,Why Do Hospitals Hate Sleep So Much?,https://news.ycombinator.com/item?id=18605709
18605412,Story,Why parking minimums almost destroyed my town and how we repealed them (2017),https://news.ycombinator.com/item?id=18605412
18604049,Story,Convert a Bird Scooter to a personal one with $32 kit,https://news.ycombinator.com/item?id=18604049
18603227,Story,Modern China Is So Crazy It Needs a New Literary Genre (2016),https://news.ycombinator.com/item?id=18603227
18602843,Story,China Announces Punishments for Intellectual-Property Theft,https://news.ycombinator.com/item?id=18602843
18601597,Story,Samsung used my DSLR photo to fake their phone’s “portrait mode”,https://news.ycombinator.com/item?id=18601597
18601588,Story,Google's cross-platform Flutter UI toolkit goes 1.0,https://news.ycombinator.com/item?id=18601588
18601515,Story,"Movim – Federated, open-source alternative to Tumblr",https://news.ycombinator.com/item?id=18601515
18600950,Story,"Announcing Open Source of WPF, Windows Forms, and WinUI",https://news.ycombinator.com/item?id=18600950
18600576,Story,"Google personalizes search results even when you’re logged out, new study finds",https://news.ycombinator.com/item?id=18600576
